In [2]:
import pandas as pd
import pyodbc
import warnings
from datetime import datetime

warnings.simplefilter('ignore')

In [6]:
FROM_DB_PATH = r"..\\..\\..\\Data\\Raw\\Sqlite\\AdventureWorks.mdb"

FROM_DB_CONN_PATH = r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=" + FROM_DB_PATH

TO_DB_CONN_PATH = r"DRIVER={SQL Server};SERVER=143.177.235.181,1433;DATABASE=AdventureWorks;UID=SA;PWD=Str0ngP@ssword"

FROM_DB = pyodbc.connect(FROM_DB_CONN_PATH)
import_cursor = FROM_DB.cursor()

TO_DB = pyodbc.connect(TO_DB_CONN_PATH)
export_cursor = TO_DB.cursor()

In [12]:
tables = ["HumanResources_Department", "HumanResources_Employee", "Person_Address", "Person_Person", "Production_BillOfMaterials","Production_Product",
          "Production_ProductCategory","Purchasing_PurchaseOrderDetail","Purchasing_PurchaseOrderHeader","Purchasing_Vendor","Sales_Customer",
          "Sales_SalesOrderDetail","Sales_SalesOrderHeader","Sales_SalesTerritory","Sales_Store"]

In [16]:
for table in tables:
    try:
        import_cursor.execute(f"SELECT * FROM {table}")
        columns = [column[0] for column in import_cursor.description]

        rows = import_cursor.fetchall()
        if not rows:
            print(f"Geen data voor {table}")
            continue

        placeholders = ", ".join(["?" for _ in columns])
        insert_query = f"INSERT INTO {table} ({', '.join(columns)}) VALUES ({placeholders})"

        export_cursor.fast_executemany = True
        export_cursor.executemany(insert_query, rows)
        TO_DB.commit()

        print(f"Tabel {table} voltooid.")

    except Exception as e:
        print(f"Fout bij tabel {table} + {e}")

Fout bij tabel HumanResources_Department + ('42S02', "[42S02] [Microsoft][ODBC SQL Server Driver][SQL Server]Invalid object name 'HumanResources_Department'. (208) (SQLExecute); [42S02] [Microsoft][ODBC SQL Server Driver][SQL Server]Statement(s) could not be prepared. (8180)")
Fout bij tabel HumanResources_Employee + ('42S02', "[42S02] [Microsoft][ODBC SQL Server Driver][SQL Server]Invalid object name 'HumanResources_Employee'. (208) (SQLExecute); [42S02] [Microsoft][ODBC SQL Server Driver][SQL Server]Statement(s) could not be prepared. (8180)")
Fout bij tabel Person_Address + ('42S02', "[42S02] [Microsoft][ODBC SQL Server Driver][SQL Server]Invalid object name 'Person_Address'. (208) (SQLExecute); [42S02] [Microsoft][ODBC SQL Server Driver][SQL Server]Statement(s) could not be prepared. (8180)")
Fout bij tabel Person_Person + ('42S02', "[42S02] [Microsoft][ODBC SQL Server Driver][SQL Server]Invalid object name 'Person_Person'. (208) (SQLExecute); [42S02] [Microsoft][ODBC SQL Server Dr